----------------
# Image Based Mechanics - hands-on session 2: FE-DIC regularization
## October, 6 - 10, 2025, CISM, Udine, Italy
#### Jean-Charles PASSIEUX, ICA/INSA Toulouse
----------------
#### We consider the same dataset as before, namely an openhole glass/epoxy coupon subjected to a tensile test. The same set of images, taken before and during the experiment, is considered again. For more details about the experiment, please refer to reference below.

#### In this session, you will try **different weak regularization techniques** and see **how to choose the penalization parameter**.

[JC. Passieux, F. Bugarin, C. David, J. Périé and L. Robert (2015)](http://dx.doi.org/10.1007/s11340-014-9872-4) **Multiscale displacement field measurement using digital image correlation: Application to the identification of elastic properties** _Experimental Mechanics_ 55(1), p.121-137

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyxel as px
import scipy.sparse as sps

## 1. Reload images, camera model and FE mesh
Images

In [13]:
filename = 'oht_cfrp_%02d.tiff'
I0 = px.Image(filename % 0).Load()
# I0.Plot()
It = px.Image(filename % 10).Load()
# It.Plot()

Mesh

In [ ]:
box = np.array([[0, 0], [90, 30]])   # outerbox
r = 5                                # hole radius
cpos = np.array([45.1, 15.3])        # position of the hole center
lc = 1                               # size of the elements close the outer boundary
lf = 0.5                               # size of the elements close to the hole
m = px.OpenHolePlateUnstructured(box, r, cpos, lc, lf)
m.Plot()

Camera model

In [16]:
cam = px.Camera(2)
p = np.array([-0.002176, 2.237317, 33.687119, 9.51087e-02])
cam.set_p(p)    
# px.PlotMeshImage(I0, m, cam)

## 2. Run FE-DIC without regularization

In [ ]:
m.Connectivity()
m.DICIntegration(cam)
U0 = px.MultiscaleInit(I0, It, m, cam, scales=[3, 2, 1])
U_noreg, res_noreg = px.Correlate(I0, It, m, cam, U0=U0)

### Plot the results without regularization

Visualization: warped mesh without regularization

In [ ]:
m.Plot(alpha=0.2)
m.Plot(U_noreg, 30)

In [ ]:
m.PlotContourDispl(U_noreg, s=30)

In [ ]:
m.PlotContourStrain(U_noreg, clim=1, cmap='RdBu')

## 3. DIC with Laplacian regularization

In [ ]:
l0 = 2  # regularization lenght in mm
L = m.Laplacian()
U_reg1, res = px.Correlate(I0, It, m, cam, U0=U0, L=L, l0=l0)

Visualization: superimpose warped meshes

In [ ]:
m.Plot(alpha=0.2)          # in gray, the reference config
m.Plot(U_reg1, 30)         # in black, regularization

### **Exercise:** play with the regularization length below from 0 (no regularisation) to $\infty$ (strong regularization)

### L-Curve for Laplacian regularization

In [ ]:
l0_all = np.array([1, 2, 5, 10, 15, 20, 30, 50])
U = U_noreg.copy()
ULU_G = [np.sqrt(U_noreg.T @ L @ U_noreg)]
rr_G = [np.sqrt(res_noreg.T @ res_noreg)]
for l0 in l0_all:
    U, res = px.Correlate(I0, It, m, cam, U0=U0, L=L, l0=l0)
    ULU_G += [np.sqrt(U.T @ L @ U)]
    rr_G += [np.sqrt(res.T @ res)]

Plot the L-Curve

In [ ]:
plt.loglog(rr_G, ULU_G, 'ko-')
for i in range(1, len(l0_all)):
    plt.text(rr_G[i], ULU_G[i], str(l0_all[i]))

### **Exercise:** choose an optimal Laplacian regularization length given the L-Curve

### Measurement with optimal penalty parameter

In [ ]:
l0_G_opt = #TODO
U_G_opt, res_G_opt = px.Correlate(I0, It, m, cam, U0=U, L=L, l0=l0_G_opt)

Visualization: warped mesh with optimal Laplacian regularization

In [ ]:
m.Plot(alpha=0.2)          # in gray, the reference config
m.Plot(U_G_opt, 30)         # in black, regularization

Plot the displacement field with optimal Laplacian regularization

In [ ]:
m.PlotContourDispl(U_G_opt, s=30)

Plot the strain field with optimal Laplacian regularization

In [ ]:
m.PlotContourStrain(U_G_opt, clim=1, cmap='RdBu')

## 4. DIC with Elastic regularization

In [ ]:
El = 20.3e3
Et = 15.4e3
nult = 0.14
Glt = 0.93e3
hooke = px.Hooke([El, Et, nult, Glt], 'orthotropic_2D')
K = m.Stiffness(hooke)
nodes_l = m.SelectEndLine('left')
nodes_r = m.SelectEndLine('right')
nodes_no_regul = np.append(nodes_l, nodes_r)
dof_no_regul = m.conn[nodes_no_regul].ravel()
D = np.ones(m.ndof)
D[dof_no_regul] = 0
Ddiag = sps.diags(D)
KDK = K @ Ddiag @ K

#### Perform DIC with the elastic regularization with an arbitrary regularization length

In [ ]:
U_reg2, res_reg2 = px.Correlate(I0, It, m, cam, U0=U_noreg, L=KDK, l0=1)

Visualization: warped meshes with elastic regularization

In [ ]:
m.Plot(alpha=0.2)          # in gray, the reference config
m.Plot(U_reg2, 30)         # in black, regularization

### **Exercise:** play with the regularization length from 0 (no regul) to $\infty$ (strong regul)

### L-Curve for elastic regularization

In [ ]:
U, res = px.Correlate(I0, It, m, cam, U0=U0)
l0_all = np.array([0.1, 0.3, 0.5, 1, 5, 10, 20, 30, 50, 75])
ULU_M = [np.sqrt(U_noreg.T @ KDK @ U_noreg)]
rr_M = [np.sqrt(res_noreg.T @ res_noreg)]
U = U_noreg.copy()
for l0 in l0_all:
    U, res = px.Correlate(I0, It, m, cam, U0=U0, L=KDK, l0=l0)
    ULU_M += [np.sqrt(U.T @ KDK @ U)]
    rr_M += [np.sqrt(res.T @ res)]

Plot the L-Curve

In [ ]:
plt.loglog(rr_M, ULU_M, 'ko-')
for i in range(1, len(l0_all)):
    plt.text(rr_M[i], ULU_M[i], str(l0_all[i]))

### **Exercise:** choose an optimal elastic regularization length given the L-Curve

In [ ]:
l0_M_opt = #TODO
U_M_opt, res_M_opt = px.Correlate(I0, It, m, cam, U0=U, L=KDK, l0=l0_M_opt)

In [ ]:
m.Plot(alpha=0.2)          # in gray, the reference config
m.Plot(U_M_opt, 30)         # in black, regularization

**Exercise.** Improve the iteration using linear prediction.

In [ ]:
m.PlotContourStrain(U_M_opt, clim=1, cmap='RdBu')